<a href="https://colab.research.google.com/github/IndigoClay/cudaCompileDeviceQuery/blob/main/cudaCompileDeviceQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Thu Mar 31 12:38:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |            

In [2]:
!git clone https://github.com/IndigoClay/cudaCompileDeviceQuery.git

Cloning into 'cudaCompileDeviceQuery'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 6), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [3]:
!pip install git+https://github.com/IndigoClay/nvcc4jupyter.git

  Cloning https://github.com/IndigoClay/nvcc4jupyter.git to /tmp/pip-req-build-77xjwnhl
  Running command git clone -q https://github.com/IndigoClay/nvcc4jupyter.git /tmp/pip-req-build-77xjwnhl
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4286 sha256=22c801c020c2e4ce0372b29798840b483c739ceacafc28f769514cd0817c3e28
  Stored in directory: /tmp/pip-ephem-wheel-cache-4pyg8qsp/wheels/c4/52/63/f7029f491c28a55cf4cb96d4abbcb3460592100edbd8560c5e
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>

#define NUM_BLOCKS 16
#define BLOCK_WIDTH 1

__global__ void hello()
{
    printf("Hello world! I'm a thread in block %d\n", blockIdx.x);
}


int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // force the printf()s to flush
    cudaDeviceSynchronize();
    
    printf("That's all!\n");

    return 0;
}

In [9]:
!cd /content/cudaCompileDeviceQuery && pwd && make

/content/cudaCompileDeviceQuery
make: Nothing to be done for 'all'.


In [ ]:
!./cudaCompileDeviceQuery/deviceQuery

In [ ]:
!pwd

In [ ]:
%%cu
#include <cuda_runtime.h>
#include <iostream>

//demonstating the usage of global memory

using namespace std;
__global__ void use_global_memory_GPU(float *array)
{
    array[threadIdx.x] = 3.14f * (float) threadIdx.x;
}

int main(int argc, char **argv)
{
    float h_arr[128];
    float *d_arr;
 
    cudaMalloc((void **)&d_arr, sizeof(float)*128);
    cudaMemcpy((void *)d_arr, (void *)h_arr, sizeof(float)*128, cudaMemcpyHostToDevice);
 
    use_global_memory_GPU<<<1, 128>>>(d_arr);
    cudaMemcpy((void *)h_arr, (void *)d_arr, sizeof(float)*128, cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
    for (auto const& i : h_arr) {
      cout << i << "  ";
    }
    cout << h_arr[2];
    //system("ls");
}